# Creación de conversaciones HTTP
Con este código se va a intentar crear todos los pasos de una conversación HTTP, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificacioness

In [1]:
import os
import torch
from transformers import (
  AutoConfig,
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  GenerationConfig,
  pipeline
)

from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [4]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
        

Your GPU supports bfloat16: accelerate training with bf16=True


In [5]:
model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1' #'mistralai/Mixtral-8x7B-Instruct-v0.1' #'mistralai/Codestral-22B-v0.1'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, legacy=True)
    
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    #device_map="auto",
    quantization_config=bnb_config,
)

generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 2048
generation_config.temperature = 0.1
generation_config.top_k = 10
generation_config.top_p = 0.1
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

model.generation_config.pad_token_ids = tokenizer.pad_token_id
    
# Crear LLM Chain
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    generation_config=generation_config,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [7]:
prompt = PromptTemplate(
    input_variables=["system", "user"],
    output_parser=None,
    partial_variables={},
    template = """
    [INST]
    {system}
    
    {user}
    [/INST]
    """
)

system_message = "You are a computer network programmer. Your goal is to generate Python code to create packages with the scapy framework based on a provided conversation summary.\n"
system_message = "For example, given the following summary of the IPv4 TCP-HTTP conversation:\n"
system_message += 'Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256 // Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"\n\n'

system_message += "x,y = random.sample(range(1, 100), 2)\n"
system_message += 'pkt1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt1[TCP].payload)+1\n'
system_message += 'pkt2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt2[TCP].payload)+1\n'
system_message += 'pkt3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt3[TCP].payload)\n'

system_message += 'pkt4= IP(src="193.24.227.230",  dst="9.9.9.9") / TCP(sport=40301, dport=80, flags="PA", seq=y, ack=x, window=8192) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webernetz.net", Path="/")\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt4[TCP].payload)\n'
system_message += 'pkt5= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y, window=4128)\n\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt5[TCP].payload)\n'
system_message += 'pkt6= IP(src="9.9.9.9",  dst="193.24.227.230") / TCP(sport=80, dport=40301, flags="PA", seq=x, ack=y, window=8192) / HTTP() / HTTPResponse(Http_Version= "HTTP/1.1", Status_Code= "200", Reason_Phrase= "OK", Content_Type="text/html") / "<HTML><BODY><H!>Hello World!</H1></BODY></HTML>"\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt6[TCP].payload)\n'
system_message += 'pkt7= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt7[TCP].payload)\n'

system_message += 'pkt8= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="FA", seq=y, ack=x, window=4128)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt8[TCP].payload)\n'
system_message += 'pkt9= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y, window=256)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt9[TCP].payload)\n'
system_message += 'pkt10= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="FA", seq=x, ack=y, window=256)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt10[TCP].payload)\n'
system_message += 'pkt11= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
system_message += 'y += len(pkt11[TCP].payload)\n'

system_message += "# Add packets to list\n"
system_message += "pktlist = [pkt1, pkt2, pkt3, pkt4, pkt5, pkt6, pkt7, pkt8, pkt9, pkt10, pkt11]"   
system_message += "\n\n"

system_message += "Variable names in the HTTP definition are extremely important, use the exact same word. For example, use Http_Version, not http_version, use Method, not method.\n\n"
system_message += "In the HTTP response, if the variable code != 200, don´t use the Content_Type variable\n"
system_message += "Use the same dport for HTTPRequest (dport=80) and sport for HTTPResponse (sport=80), as this means we are creating a HTTP conection and using another port like 22 might make it seem like SSH traffic\n\n"
system_message += 'Pay special attention to the sequence and ack numbers so they make sense for a TCP conversation. The must be continuous throgh the conversation. Here you have a little explanation of how they work:\n'
system_message += 'The Sequence Number (SEQ) marks the starting byte of each segment sent. It increments by the number of bytes in the payload of each outgoing segment, reflecting the continuous flow of data.\n'
system_message += 'The Acknowledgment Number (ACK) acknowledges the receipt of data. It corresponds to the next expected byte the receiver anticipates from the sender. As segments arrive at the receiver, the ACK number adjusts to reflect the highest contiguous sequence number received.\n'
system_message += 'So, SEQ numbers increase with outgoing data, while ACK numbers advance with incoming acknowledgments, ensuring synchronized data exchange in a TCP connection.\n\n'   

system_message += "Take your time (few seconds) to generate the commands and compare your proposed solution with real HTTP functioning, if anything is different, change it in order to do as real HTTP\n\n"
system_message += "\n\n"

In [8]:
with open("./data/Conversations/HTTP/Conv_summaries_vfin.txt","r",encoding="utf8") as f:
    Conv_summaries = f.read().splitlines()

print("Número de resúmenes: " +str(len(Conv_summaries)))

Número de resúmenes: 65


In [9]:
class code_response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [10]:
import pickle

try:
    responses = pickle.load(open("./data/Conversations/HTTP/pickle/HTTP_pairs_Aday.pkl", "rb" ))
except:
    responses = []
    
print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 0


In [11]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
rag_chain = prompt | mistral_llm

/home/ivangm/pytorch_env/pytorch_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [12]:
from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.inet import IP, TCP
from scapy.layers.http import *

from tqdm.auto import tqdm

progress_bar = tqdm(range(len(Conv_summaries)))

counter = 0

for i in range(len(Conv_summaries)):

    text_sum = ""
    
    summary = Conv_summaries[i]
    
    #print("\n...........................................")
    #print("GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")\n")
    
    query_content = "Given the following summary of a conversation:\n"
    query_content += summary + "\n"
    text_sum += summary + "\n"
    query_content += "# Instructions:\n"
    query_content += "## Generate a complete python code for creating packets with scapy framework. Use the provided example structure to simulate real-world conditions, adding the len and sleep function calls between packets.\n"
    query_content += "## Take your time (a few seconds) to validate if the last code line of your code is 'pktlist = [pkt1, pkt2,...]'; If not, make the appropriate corrections to the code.\n"
    query_content += "## Don't explain the code, just generate the code block itself. PLEASE DONT start the responses with ```python. This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output without ```python please.\n"

    # Query es el mensaje que le envias al modelo
    #print(query_content)

    completion = ""
    completion = rag_chain.invoke({"system": system_message, "user": query_content})

    progress_bar.update(1)

    #print("\n...Generated!")

    # Filtramos ```python
    #pos1 = completion.find("```python\n")
    #pos2 = completion.find("\n```")
    #print(pos1, pos2)
    #if pos1 != -1 and pos2 != 1:
    #    completion = completion[pos1+len("```python\n"):pos2]

    # Parece que hay problemas de indentado en el código resultante ¿?
    completion_copy = completion
    completion = ""
    for line in completion_copy.splitlines():
        completion += line.strip()+"\n"
        
    # Guardamos tanto la petición como la respuesta en un archivo
    #pickle.dump(responses, open( "./data/Conversations/ARP/pickle/ARP_pairs_Aday.pkl", "wb" ) )
    try:
        #print("Ejecutando python...")
        exec(completion)
        #print(pktlist)
        counter += 1
    except:
        print("Error en la ejecución\n")
        #print(completion)
        #print(":".join("{:02x}".format(ord(c)) for c in completion))
        continue

    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    try:
        responses.append(code_response(text_sum, completion))
    except:
        print("Error almacenando resultado del modelo\n")
    
    # Guardamos los paquetes generados en un pcap
    with open("./data/Conversations/HTTP/pcap/HTTP_generated_mixtral.pcap", "ba+") as f:
        wrpcap(f, pktlist, append=True)

print("Number of completitions done: " + str(counter))


  0%|          | 0/65 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error en la ejecución



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error en la ejecución



...

Error en la ejecución



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error en la ejecución



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error en la ejecución

Number of completitions done: 23


In [13]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion)

Source: IP="192.168.0.10", port=4592, Window: 6553 // Destination: IP="192.168.0.11", port=80, Window: 3276 // Others: Host="http://www.bbc.co.uk/", Path="", Code=200,  Reason_Phrase= "OK"


from scapy.all import *
import time
import random

summary = {
"source": {"ip": "192.168.0.10", "port": 4592, "window": 6553},
"destination": {"ip": "192.168.0.11", "port": 80, "window": 3276},
"others": {"host": "http://www.bbc.co.uk/", "path": "", "code": 200, "reason\_phrase": "OK"}
}

x, y = random.sample(range(1, 100), 2)

pkt1 = IP(src=summary["source"]["ip"], dst=summary["destination"]["ip"]) / TCP(sport=summary["source"]["port"], dport=summary["destination"]["port"], flags="S", seq=y, window=summary["source"]["window"])
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt1[TCP].payload) + 1

pkt2 = IP(src=summary["destination"]["ip"], dst=summary["source"]["ip"])/ TCP(sport=summary["destination"]["port"], dport=summary["source"]["port"], flags="SA", seq=x, ack=y, window=summary["destination"